# Scheme Management

```{seealso}
The Vast DB SDK API Documentation is available [here](https://vastdb-sdk.readthedocs.io).
```

## Introduction

This notebook provides a comprehensive guide to managing schemas within your VAST Database using the Python SDK. We'll cover essential functions like creating new schemas, listing existing ones, retrieving specific schemas, renaming schemas, and deleting them.

By following along, you'll gain the ability to effectively organize and access your data within VAST DB. The code examples demonstrate each function's usage with clear explanations, making it easy to integrate these operations into your workflow.


## Install sdk and connect to Vast DB

Install vastdb library.

In [ ]:
!pip install --quiet vastdb

In [ ]:
# This cell is used for automated testing.  It will be removed when published.

import os

ENDPOINT = os.environ['ENDPOINT']
ACCESS_KEY = os.environ['ACCESS_KEY']
SECRET_KEY = os.environ['SECRET_KEY']
DATABASE_NAME = os.environ['DATABASE_NAME']
DATABASE_SCHEMA = os.environ['DATABASE_SCHEMA']

In [ ]:
# Change these variables to reflect your environment, E.g. 
#
# ENDPOINT = 'http://your_vast_endpoint:12345'
# DATABASE_NAME = 'your_db'
# ACCESS_KEY = 'your_access_key'
# SECRET_KEY = 'your_secret_key'
#
# This will get created:
# DATABASE_SCHEMA = 'your_database_schema'

Connect to Vast DB:

In [ ]:
import vastdb

session = vastdb.connect(
    endpoint=ENDPOINT,
    access=ACCESS_KEY,
    secret=SECRET_KEY)

## Schema Management API

### `create_schema`
- **Usage**: Create a new schema (a container of tables) in a bucket..
- **Parameters**:
  - `name` (str): Name of the schema to create.
  - `fail_if_exists` (bool, optional, default=True): If True, fail if the schema name already exists in the bucket


In [9]:
# Note that we use the fail_if_exists=False so that the command doesn't
# return an error if the schema already exists
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    schema = bucket.create_schema(DATABASE_SCHEMA, fail_if_exists=False)
    print(f"Schema: {schema.name}")

Schema: python-sdk-schema


### `schemas`
- **Usage**: List all schemas in a bucket.
- **Parameters**:
  - `batch_size` (int, optional): Maximum number of schemas to retrieve (default is `1000`).

In [ ]:
# Print all schemas in the Database
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    for schema in bucket.schemas():
        print(schema)

### `schema`
- **Usage**: Get a specific schema (a container of tables) under this schema.
- **Parameters**:
  - `name` (str): Name of the schema to get.
  - `fail_if_missing` (bool, optional, default=True): If True, fail with an exception if the schema doesn't exist, else return None


In [ ]:
schema_name='python-sdk1'

with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    try:
        # here we select the schema we are interested in
        schema = bucket.schema(name=DATABASE_SCHEMA)
        print(schema)
    except Exception as e:
        print("Schema doesn't exist:", e)

### `rename`
- **Usage**: Rename this schema.
- **Parameters**:
  - `new_name` (str): New name for the schema.

In [ ]:
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    schema = bucket.create_schema(
        f'{DATABASE_SCHEMA}_TEMPORARY_SCHEMA', 
        fail_if_exists=False)
    
    print(f"Schema: {schema.name}")

In [ ]:
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    try:
        schema = bucket.schema(
            name=f'{DATABASE_SCHEMA}_TEMPORARY_SCHEMA', 
            fail_if_missing=False)
        
    except Exception as e:
        print("Schema doesn't exist:", e)

    # if the schema exists, we can rename it
    if schema:
        try:
            schema.rename(new_name=f'{DATABASE_SCHEMA}_TEMPORARY_SCHEMA')
            print("Schema renamed")
        except Exception as e:
            print("Unable to rename schema:", e)

In [ ]:
# verify the schema exists
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    schema = bucket.schema(name='TEMPORARY_SCHEMA_RENAMED')
    print(schema)

### `drop`
- **Usage**: Delete a schema in a bucket.
- **Parameters**:
  - No parameters

In [ ]:
# let's drop the schma we created in the previous step
with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)

    # first retrieve the schema
    try:
        schema = bucket.schema(
            name=f'{DATABASE_SCHEMA}_TEMPORARY_SCHEMA', 
            fail_if_missing=False)
        
        print(schema)
    except Exception as e:
        print("Schema doesn't exist:", e)

    # then drop it
    if schema:
        try:
            schema.drop()
            print("Schema dropped")
        except Exception as e:
            print("Unable to drop schema:", e)

In [ ]:
# verify the schema names

with session.transaction() as tx:
    bucket = tx.bucket(DATABASE_NAME)
    for schema in bucket.schemas():
        print(schema)